In [7]:
import pandas as pd

df = pd.read_csv("../datasets/Google.csv")
df.index = pd.DatetimeIndex(df['Date'].values)

In [8]:
df.head()

,Date,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
2004-08-19,2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [9]:
df.index - pd.Timedelta(days=1)

DatetimeIndex(['2004-08-18', '2004-08-19', '2004-08-22', '2004-08-23',
               '2004-08-24', '2004-08-25', '2004-08-26', '2004-08-29',
               '2004-08-30', '2004-08-31',
               ...
               '2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05',
               '2017-01-08', '2017-01-09', '2017-01-10', '2017-01-11',
               '2017-01-12', '2017-01-16'],
              dtype='datetime64[ns]', length=3125, freq=None)

In [14]:
df["Close"].pct_change()

2004-08-19         NaN
2004-08-20    0.079484
2004-08-23    0.010064
2004-08-24   -0.041408
2004-08-25    0.010775
2004-08-26    0.018019
2004-08-27   -0.016310
2004-08-30   -0.039001
2004-08-31    0.003529
2004-09-01   -0.020709
2004-09-02    0.012569
2004-09-03   -0.014777
2004-09-07    0.015698
2004-09-08    0.007088
2004-09-09    0.000098
2004-09-10    0.029518
2004-09-13    0.020602
2004-09-14    0.037116
2004-09-15    0.004574
2004-09-16    0.017589
2004-09-17    0.030885
2004-09-20    0.015916
2004-09-21   -0.012735
2004-09-22    0.004582
2004-09-23    0.020612
2004-09-24   -0.008194
2004-09-27   -0.013102
2004-09-28    0.072721
2004-09-29    0.033265
2004-09-30   -0.011291
                ...   
2016-12-02    0.000170
2016-12-05    0.018000
2016-12-06   -0.002621
2016-12-07    0.019699
2016-12-08    0.004675
2016-12-09    0.017958
2016-12-12   -0.001915
2016-12-13    0.009209
2016-12-14    0.003128
2016-12-15   -0.002739
2016-12-16   -0.007123
2016-12-19    0.003285
2016-12-20 

In [ ]:
df.index.se

In [15]:
help(df.index.searchsorted)

Help on method searchsorted in module pandas.core.indexes.datetimes:

searchsorted(value, side='left', sorter=None) method of pandas.core.indexes.datetimes.DatetimeIndex instance
    Find indices where elements should be inserted to maintain order.
    
    Find the indices into a sorted DatetimeIndex `self` such that, if the
    corresponding elements in `value` were inserted before the indices,
    the order of `self` would be preserved.
    
    Parameters
    ----------
    value : array_like
        Values to insert into `self`.
    side : {'left', 'right'}, optional
        If 'left', the index of the first suitable location found is given.
        If 'right', return the last such index.  If there is no suitable
        index, return either 0 or N (where N is the length of `self`).
    sorter : 1-D array_like, optional
        Optional array of integer indices that sort `self` into ascending
        order. They are typically the result of ``np.argsort``.
    
    Returns
    ----

In [87]:
import numpy as np
import pandas as pd

def getDailyVol(close, span=100):
    use_idx = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    use_idx = use_idx[use_idx > 0]
    # Get rid of duplications in index
    use_idx = np.unique(use_idx)
    prev_idx = pd.Series(close.index[use_idx - 1], index=close.index[use_idx])
    ret = close.loc[prev_idx.index] / close.loc[prev_idx.values].values - 1
    vol = ret.ewm(span=span).std()
    return vol

In [91]:
vol = getDailyVol(df["Close"])
vol.head()

2004-08-23         NaN
2004-08-24    0.036396
2004-08-25    0.029930
2004-08-26    0.027366
2004-08-30    0.029365
Name: Close, dtype: float64

In [102]:
def applyPtSlOnTl(close, events, ptSl, molecule):
    _events = events.loc[molecule]
    out = _events[['tl']].copy(deep=True)
    # Set Profit Taking and Stop Loss
    if ptSl[0] > 0:
        pt = ptSl[0] *  _events["trgt"]
    else:
        pt = pd.Series(index=_events.index)
    if ptSl[1] > 0:
        sl = ptSl[1] * _events["trgt"]
    else:
        sl = pd.Series(index=_events.index)
    for loc, tl in events_["tl"].fillna(close.index[-1]).iteritem():
        df = close[loc:tl]
        df = (df / close[loc] - 1) * _events.at[loc, 'side']
        out.loc[loc, 'sl'] = df[df < sl[loc]].index.min()
        out.loc[loc, 'pt'] = df[df > pt[loc]].index.min()
    return out

In [89]:
ret

2004-08-23         NaN
2004-08-24    0.036396
2004-08-25    0.029930
2004-08-26    0.027366
2004-08-30    0.029365
2004-08-31    0.026605
2004-09-01    0.024794
2004-09-02    0.024097
2004-09-07    0.023610
2004-09-08    0.022400
2004-09-09    0.021152
2004-09-13    0.021182
2004-09-14    0.022876
2004-09-15    0.021867
2004-09-16    0.021314
2004-09-20    0.020706
2004-09-21    0.020498
2004-09-22    0.019782
2004-09-23    0.019554
2004-09-27    0.019453
2004-09-28    0.024991
2004-09-29    0.024968
2004-09-30    0.024733
2004-10-04    0.024173
2004-10-05    0.023793
2004-10-06    0.023561
2004-10-07    0.022996
2004-10-11    0.023169
2004-10-12    0.022692
2004-10-13    0.022470
                ...   
2016-11-21    0.011895
2016-11-22    0.011777
2016-11-25    0.011660
2016-11-28    0.011579
2016-11-29    0.011476
2016-11-30    0.011646
2016-12-01    0.011727
2016-12-05    0.011873
2016-12-06    0.011764
2016-12-07    0.011951
2016-12-08    0.011844
2016-12-12    0.011733
2016-12-13 

In [94]:
np.power(2, 0.33333)

1.2599181388624912

In [95]:
155 * 1.25

193.75

In [101]:
ret.at["2004-08-24 "]

0.03639576485717827